In [9]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy
import ipywidgets
from ipywidgets import Output, Button, widgets, Layout
from IPython.display import HTML, clear_output, display
from base64 import b64encode

import cryspy
from cryspy.B_parent_classes.cl_4_global import get_table_html_for_variables
from cryspy.H_functions_global.powder_experiments import report_powder_experiments
from cryspy.A_functions_base.database import DATABASE
from cryspy.A_functions_base.charge_form_factor import get_atom_name_ion_charge_shell, get_shell_orbit_electrons, calc_jl_for_shell_of_ion
from cryspy.A_functions_base.magnetic_form_factor import calc_j0, calc_j2

In [10]:
D_ELEMENTS = DATABASE["Elements"]
D_MFF = DATABASE["Magnetic Form Factor, tabulated"]
D_ISOTOPES = DATABASE["Isotopes"]
D_SA = DATABASE["Scattering amplitude"]
D_SO = DATABASE["Orbitals by radial Slater functions"]
D_DISPERSION = DATABASE["Dispersion"]

In [4]:
L_ELEMENTS = [hh[1] for hh in D_ELEMENTS.keys() if hh[0].startswith("Atomic number")]

In [5]:
def get_info_about_element(type_symbol:str, shells: list):
    element, ion_charge, full_shell = get_atom_name_ion_charge_shell(type_symbol)
    l_key = ["Atomic number", "Classification", "Full_name", "Orbital occupancy", "Atomic weight", "b_scat_Re", "b_scat_Im"]
    sthovl = numpy.linspace(0,2,101)
    d_info = {"sthovl": sthovl}
    l_key_element = D_ELEMENTS.keys()
    for name in l_key:
        key_element = (name, element)
        if key_element in l_key_element:
            d_info[name]=D_ELEMENTS[key_element]

    kappa = 1.
    
    
    if len(shells) > 0:
        full_valence_shell = " ".join([f"{hh:}1" for hh in shells])
        try:
            mff_jl_rad = calc_jl_for_shell_of_ion(sthovl, type_symbol, full_valence_shell)
            d_info["mff_jl_rad"] = mff_jl_rad
        except UserWarning:
            pass

    l_key_mff = D_MFF.keys()
    key_mff = ("j0_AaBbCcD", element, ion_charge)
    if key_mff in l_key_mff:
        j0_parameters = D_MFF[key_mff]
        mff_j0_tab = calc_j0(sthovl, kappa, j0_parameters)[0]
        d_info["mff_j0_tab"] = mff_j0_tab
    key_mff = ("j2_AaBbCcD", element, ion_charge)
    if key_mff in l_key_mff:
        j2_parameters = D_MFF[key_mff]
        mff_j2_tab = calc_j2(sthovl, kappa, j2_parameters)[0]
        d_info["mff_j2_tab"] = mff_j2_tab
    return d_info

In [28]:
#  background="logo.png"
display(HTML("""
<hr>
<form width: 100%; align="center">
<a href="https://sites.google.com/view/cryspy/" title="CrysPy site"><img src="logo.png" alt="CrysPy logo" width="400"></a>
</form>
<hr>
<p align="center"><b>Web application</b> of the crystallographic python library CrysPy.
The library can be used as a handbook (click on the <i>"Handbook"</i> tab to start).
Upload your ".rcif" file or begin with one of the examples by selecting <i>"Upload File"</i> to start analyzing the diffraction data.
</p>
<hr>
"""))

In [6]:
# Reference: https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
import requests
import io
def download_file_from_google_drive(id):
    destination = io.BytesIO()
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)
    s_content = destination.getvalue().decode("ASCII")
    return s_content 

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    for chunk in response.iter_content(CHUNK_SIZE):
        if chunk: # filter out keep-alive new chunks
            destination.write(chunk)

In [7]:
# global parameters
DDATA = {
    "rcif_obj": cryspy.str_to_globaln("global_"), 
    "file_rcif_name": "out.rcif"
}
L_EXAMPLE = [
    ("PND: Single crystal", "rhochi_single_susceptibility_Ho2Ti2O7.rcif", "1xG6Dz7f78XinH78uPNu0Mo6eU8w5Lh0L"),
    ("PND: Powder 1D", "rhochi_polarized_powder_1d_Ho2Ti2O7_5K_1T.rcif", "18qdPs558CN39ABkR3JqtfNgvpn4SgCQo"),
    ("PND: Powder 2D", "rhochi_polarized_powder_2d_DyAl_5K_5T.rcif", "1GKnDsdKbYW6vhXuNQaS_jn1SddB6Viul"),
    ("Unpolarized neutron diffraction", "rhochi_unpolarized_powder_1d_PbSO4.rcif", "1wax6x5wnoFiQXArTWbWv-E3YTWUKF6EV"),
    ("X-ray diffraction", "rhochi_unpolarized_powder_1d_xrays_PbSO4.rcif", "1eQZLb613h7c0J5QA8fLlqhUCW6tI_-06"),
    ("TOF neutron diffraction", "rhochi_unpolarized_tof_powder_CeCuAl.rcif", "1qflJs3uXdG-O35rTGyyTXEqRa9WKELmX"),
    ("Spin density (MEM)", "mempy_spin_density_YTiO3.rcif", "1S8YPVcNyYKEkP8j4RkUB7X9vn85bm435"),
    ("Magnetization density (MEM)", "mempy_magnetization_density_Yb2Ti2O7_2K_1T.rcif", "1PQmBK0xDu7Zxs-QzmFAzX0HtJnGMQkdY"),
    ]


In [8]:
# global widgets

W_OUT = widgets.Output()

In [9]:
LAYOUT_15 = Layout(
    display='flex',    
    justify_content='center',
    # border='solid 2px',
    align_items='stretch',
    width='15%',
)
LAYOUT_70 = Layout(
    display='flex',    
    justify_content='center',
    # border='solid 2px',
    align_items='stretch',
    width='70%',
)

LAYOUT_100 = Layout(
    display='flex',
    flex_flow='row',
    justify_content='center',
    # border='solid 2px',
    align_items='stretch',
    width='99%',
    # height="400px",
)


In [10]:
# page MENU
def draw_page_menu():
    widget_fo = widgets.FileUpload(
        description = "Upload RCIF",
        accept='.rcif',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
        multiple=False,  # True to accept multiple files upload else False
    )

    def change_output(ddata):
        s_cont = ddata["new"][0]["content"].tobytes().decode("ASCII")
        DDATA["rcif_obj"] = cryspy.str_to_globaln(s_cont)
        W_OUT.clear_output()
        with W_OUT:
            print(ddata["new"][0]["name"])
            DDATA["file_rcif_name"]=ddata["new"][0]["name"]
            print("File was uploaded")
        DISPLAY_NEW_OBJECT()
    
    widget_fo.observe(change_output, names="value")
    
    def trigger_download(text, filename, kind='text/json'):
        # see https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URIs for details
        content_b64 = b64encode(text.encode()).decode()
        data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
        js_code = f"""
            var a = document.createElement('a');
            a.setAttribute('download', '{filename}');
            a.setAttribute('href', '{data_url}');
            a.click()
        """
        with W_OUT:
            clear_output()
            display(HTML(f'<script>{js_code}</script>'))


    
    btn_download = Button(description='Download it to PC')
    
    def download_rcif_file(e=None):
        file_name = DDATA["file_rcif_name"]
        if DDATA["rcif_obj"] is not None:
            trigger_download(DDATA["rcif_obj"].to_cif(), file_name, kind='text/plain')
        else:
            with W_OUT:
                print("File is not specified")
        
    btn_download.on_click(download_rcif_file)

    page_menu = widgets.Box(
        [widget_fo, btn_download],
        layout=Layout(justify_content="center"))
    return page_menu

In [11]:
# page object
def draw_page_object():
    nrows = 10
    select_data_item = widgets.Select(
        options=[],
        description='',
        rows=nrows,
        tooltip="Select a data block",
        disabled=False,
        tabbable=True,
        layout=LAYOUT_100,
        )
    select_loop_item = widgets.Select(
        options=[],
        description='',
        disabled=True,
        tooltip="Select an item of the selected  data block",
        rows=nrows,
        layout=LAYOUT_100,
        )

    cif_text = widgets.Textarea(
        value='',
        placeholder="""Presentation of Item in STAR format""",
        description='',
        disabled=False, # True
        rows=nrows,
        layout=LAYOUT_100,
    )
    
    button_save_changes = widgets.Button(
        description="Save changes of current item", 
        layout=Layout(justify_content="center", width="99%"))

    button_add_to_datas = widgets.Button(
        description="Add Item to data blocks", 
        layout=Layout(justify_content="center", width="99%"))
    
    button_add_to_items = widgets.Button(
        description="Add Item to items", 
        layout=Layout(justify_content="center", width="99%"))

    button_del_data = widgets.Button(
        description="Delete the selected data block", 
        layout=Layout(justify_content="center", width="99%"))
    
    button_del_item = widgets.Button(
        description="Delete the selected item", 
        layout=Layout(justify_content="center", width="99%"))

    def func_loop(dinfo_data_item):
        data_index = select_data_item.index
        data_item = DDATA["rcif_obj"].items[data_index]
        W_OUT.clear_output()
        with W_OUT:
            plt.close()
            l_plots = data_item.plots()
            if len(l_plots)>0:
                plt.show()            
            display(HTML(data_item.report_html()))
            # display(HTML(data_item._repr_html_()))
        if isinstance(data_item, cryspy.DataN):
            l_loop_item_name = [item.get_name() for item in data_item]
            select_loop_item.disabled = False
            select_loop_item.options = l_loop_item_name
            select_loop_item.index = None
            cif_text.value = ""
            cif_text.disabled = False # True
        else:
            cif_text.value = data_item.to_cif()
            select_loop_item.options = ()
            select_loop_item.disabled = True
            cif_text.disabled = False
        return
    
    select_data_item.observe(func_loop, names="index")
    
    def show_cif(dinfo_loop_item):
        data_index = select_data_item.index
        data_item = DDATA["rcif_obj"].items[data_index]
        loop_index = select_loop_item.index
        if loop_index is None:
            cif_text.value = data_item.to_cif()
            selected_item = data_item
        else:
            loop_item = data_item.items[loop_index]
            cif_text.value = loop_item.to_cif()
            selected_item = loop_item
        cif_text.disabled = False
        W_OUT.clear_output()
        with W_OUT:
            plt.close()
            l_selected_item_plots = selected_item.plots()
            if len(l_selected_item_plots)>0:
                plt.show()
            display(HTML(selected_item.report_html()))
            # display(HTML(loop_item._repr_html_()))

        
    select_loop_item.observe(show_cif, names="index")
    
    def save_cif(b):        
        data_index = select_data_item.index
        data_item = DDATA["rcif_obj"].items[data_index]
        loop_index = select_loop_item.index
        
        globaln_new = cryspy.str_to_globaln(cif_text.value)
        if len(globaln_new.items) != 0:
            loop_item_new = globaln_new.items[0]
            if isinstance(loop_item_new, (cryspy.LoopN, cryspy.ItemN)):
                if loop_index is None:
                    DDATA["rcif_obj"].items.pop(data_index)
                    DDATA["rcif_obj"].items.insert(data_index, loop_item_new)
                else:
                    data_item.items.pop(loop_index)
                    data_item.items.insert(loop_index, loop_item_new)
                show_cif({})

    def add_to_globaln(b):        
        data_index = select_data_item.index
        globaln_new = cryspy.str_to_globaln(cif_text.value)
        if len(globaln_new.items) != 0:
            l_data_loop_item_new = globaln_new.items
            for data_loop_item_new in l_data_loop_item_new:
                if isinstance(data_loop_item_new, (cryspy.LoopN, cryspy.ItemN, cryspy.DataN)):
                    DDATA["rcif_obj"].items.append(data_loop_item_new)
            DISPLAY_NEW_OBJECT()

    def add_to_datan(b):        
        data_index = select_data_item.index
        l_data_item = DDATA["rcif_obj"].items[data_index]
        globaln_new = cryspy.str_to_globaln(cif_text.value)
        if len(globaln_new.items) != 0:
            l_loop_item_new = globaln_new.items
            for loop_item_new in l_loop_item_new:
                if isinstance(loop_item_new, (cryspy.LoopN, cryspy.ItemN)):
                    l_data_item.items.append(loop_item_new)
            DISPLAY_NEW_OBJECT()
    
    def del_data(b):        
        data_index = select_data_item.index
        if data_index is not None:
            DDATA["rcif_obj"].items.pop(data_index)
            DISPLAY_NEW_OBJECT()

    def del_item(b):        
        data_index = select_data_item.index
        if data_index is not None:
            l_data_item = DDATA["rcif_obj"].items[data_index]
            loop_index = select_loop_item.index
            if loop_index is not None:
                l_data_item.items.pop(loop_index)
                DISPLAY_NEW_OBJECT()


    button_save_changes.on_click(save_cif)
    button_add_to_datas.on_click(add_to_globaln)
    button_add_to_items.on_click(add_to_datan)
    button_del_data.on_click(del_data)
    button_del_item.on_click(del_item)
    
    items = [
        widgets.Label("Data blocks (click to select):"), widgets.Label("Items of selected data block:"), widgets.Label("Selected Item:"), 
        select_data_item, select_loop_item,cif_text, 
        button_del_data, button_del_item, button_save_changes,
        button_add_to_datas,button_add_to_items,
        ]
    # page_object = widgets.HBox(
    #     items, 
    #     layout=Layout(flex_flow="row wrap", justify_content='center'))
        
    page_object =widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2, 15%) 70%"))

    return page_object, select_data_item, select_loop_item, cif_text

In [12]:
def draw_page_cryspy_methods():


    button_info = widgets.Button(description="Get info")
    button_calculations = widgets.Button(description="Compute")
    button_fitting = widgets.Button(description="Refinement")
    button_mempy_sd = widgets.Button(description="Spin density (MEM)")
    button_mempy_md = widgets.Button(description="Magn. density (MEM)")

    items = [button_info, button_calculations, button_fitting, button_mempy_sd, button_mempy_md]
    
    def disabled_buttons(flag:bool=False):
        for button in items:
            button.disabled = flag

    def print_object_info(b):
        disabled_buttons(True)
        rcif_object = DDATA["rcif_obj"]
        
        ls_html = []
        ls_html.append(get_table_html_for_variables(rcif_object))
        ls_html.append(rcif_object.report_html())
        s_powder = report_powder_experiments(DDATA["rcif_obj"])
        W_OUT.clear_output()
        with W_OUT:
            display(HTML("\n".join(ls_html)))
            print(s_powder)
        disabled_buttons(False)
        return 

    def run_fitting(b):
        disabled_buttons(True)
        W_OUT.clear_output()
        with W_OUT:
            print("Calculations are running...")
            cryspy.rhochi_rietveld_refinement(DDATA["rcif_obj"])
            print("Calculations completed.")
            plt.close()
            l_plots = DDATA["rcif_obj"].plots()
            if len(l_plots)>0:
                plt.show()            
            DISPLAY_NEW_OBJECT()
        disabled_buttons(False)

    def run_chi_sq(b):
        disabled_buttons(True)
        W_OUT.clear_output()
        with W_OUT:
            print("Calculations are running...")
            cryspy.rhochi_no_refinement(DDATA["rcif_obj"])
            print("Calculations completed.")
            plt.close()
            l_plots = DDATA["rcif_obj"].plots()
            if len(l_plots)>0:
                plt.show()
            DISPLAY_NEW_OBJECT()
        disabled_buttons(False)

    def run_mempy_sd(b):
        disabled_buttons(True)
        W_OUT.clear_output()
        with W_OUT:
            print("Calculations are running...")
            cryspy.mempy_spin_density_reconstruction(DDATA["rcif_obj"])
            print("Calculations completed.")
            plt.close()
            l_plots = DDATA["rcif_obj"].plots()
            if len(l_plots)>0:
                plt.show()
            DISPLAY_NEW_OBJECT()
        disabled_buttons(False)

    def run_mempy_md(b):
        disabled_buttons(True)
        W_OUT.clear_output()
        with W_OUT:
            print("Calculations are running...")
            cryspy.mempy_magnetization_density_reconstruction(DDATA["rcif_obj"])
            print("Calculations completed.")
            plt.close()
            l_plots = DDATA["rcif_obj"].plots()
            if len(l_plots)>0:
                plt.show()
            DISPLAY_NEW_OBJECT()
        disabled_buttons(False)

    button_info.on_click(print_object_info)
    button_fitting.on_click(run_fitting)
    button_calculations.on_click(run_chi_sq)
    button_mempy_sd.on_click(run_mempy_sd)
    button_mempy_md.on_click(run_mempy_md)

    page_cryspy_methods =  widgets.Box(
        items,
        layout=Layout(flex_flow="row wrap", justify_content='center'))

    # accordion = widgets.Accordion(
    #         children=[page_cryspy_methods, ],
    #         titles=('Methods',))

    return page_cryspy_methods

In [13]:
def draw_page_examples():
    btn_1 = Button(description=L_EXAMPLE[0][0], tooltip="Ho2Ti2O7")
    btn_2 = Button(description=L_EXAMPLE[1][0], tooltip="Ho2Ti2O7")
    btn_3 = Button(description=L_EXAMPLE[2][0], tooltip="DyAl garnet")
    btn_4 = Button(description=L_EXAMPLE[3][0], tooltip="PbSO4")
    btn_5 = Button(description=L_EXAMPLE[4][0], tooltip="PbSO4")
    btn_6 = Button(description=L_EXAMPLE[5][0], tooltip="CeCuAl")
    btn_7 = Button(description=L_EXAMPLE[6][0], tooltip="YTiO3")
    btn_8 = Button(description=L_EXAMPLE[7][0], tooltip="Yb2Ti2O7")

    def download_example(b):
        l_name = [hh[0] for hh in L_EXAMPLE]
        ind_example = l_name.index(b.description)
        file_id = L_EXAMPLE[ind_example][2]
        s_content = download_file_from_google_drive(file_id)
        
        DDATA["rcif_obj"]=cryspy.str_to_globaln(s_content)
        DDATA["file_rcif_name"] = L_EXAMPLE[ind_example][1]
        DISPLAY_NEW_OBJECT()
        return 

    btn_1.on_click(download_example)
    btn_2.on_click(download_example)
    btn_3.on_click(download_example)
    btn_4.on_click(download_example)
    btn_5.on_click(download_example)
    btn_6.on_click(download_example)
    btn_7.on_click(download_example)
    btn_8.on_click(download_example)

    w_examples = widgets.Box(
            [btn_1, btn_2, btn_3, btn_4, btn_5, btn_6, btn_7, btn_8],
            layout=Layout(
                    flex_flow="row wrap", 
                    justify_content="center"))
    # accordion = widgets.Accordion(
    #         children=[w_examples, ],
    #         titles=('.. or use one of the examples',))
    # page_examples = widgets.Box(
    #     [accordion, ],
    #     layout=Layout(justify_content="center"))
    return w_examples

In [16]:
# page handbook
def draw_page_handbook():
    
    tags = widgets.TagsInput(
        value=[],
        placeholder="Symbol of chemical element",
        tooltip="Type only the chemical symbol: 'O', 'Fe'; or together with the oxygen state: 'Fe3+', 'O2-', or together with the atomic shell:  'Fe3+ 3d', 'O2- 2p'.",
        
        allow_duplicates=False
    )
    
    def get_info_about_elements(b):
        ls_html = ["<table><tr><th>Chemical symbol</th>"]
        l_d_info = []
        flag_mff = False
        for input_string in tags.value:
            flag = True
            l_input_param = input_string.strip().split()
            type_symbol = l_input_param[0].capitalize()
            shells = l_input_param[1:]
            try:
                element, ion_charge, full_shell = get_atom_name_ion_charge_shell(type_symbol)
            except KeyError:
                flag=False
            if flag:
                d_info = get_info_about_element(type_symbol, shells=shells)
                d_info["Type symbol"] = type_symbol
                flag_mff = (flag_mff | (
                    ("mff_jl_rad" in d_info.keys()) | ("mff_j0_tab" in d_info.keys())
                    ))
                d_info["Element"] = element
                d_info["Shell"] = " ".join(shells)
                d_info["Oxygen state"] = ion_charge
                d_info["Electron configuration"] = d_info["Orbital occupancy"]
                if ion_charge != 0:
                    if len(full_shell.strip().split()) > 3:
                        full_shell = "... "+" ".join(full_shell.strip().split()[-3:])
                        d_info["Electron configuration"] = full_shell
                l_d_info.append(d_info)
                ls_html.append(f"<th>{d_info['Element']:}</th>")
        ls_html.append("</tr>")
        l_table = ["Full_name", "Classification", "Atomic number", "Oxygen state", "Electron configuration", "Atomic weight", "b_scat_Re", "b_scat_Im"]
        for name_table in l_table:
            ls_html.append(f"<tr><th>{name_table:}</th>")
            for d_info in l_d_info:
                if name_table in d_info.keys():
                    ls_html.append(f"<td>{d_info[name_table]:}</td>")
                else:
                    ls_html.append(f"<td></td>")
            ls_html.append("</tr>")
        ls_html.append("</table>")
        
        W_OUT.clear_output()
        plt.close()
        with W_OUT:
            display(HTML("\n".join(ls_html)))
            if flag_mff:
                fig, ax = plt.subplots()
                ax.set_xlabel(r"sin $\theta$ / $\lambda$ ($\AA^{-1}$)")
                ax.set_ylabel("magnetic form-factor")
                for d_info in l_d_info:
                    
                    if ("mff_jl_rad" in d_info.keys()):
                        sthovl = d_info["sthovl"]
                        mff_jl_rad = d_info["mff_jl_rad"]
                        ax.plot(sthovl, mff_jl_rad[:, 0], "-", label=f"<j0> {d_info['Type symbol']:} {d_info['Shell']:}", alpha=0.7)
                        ax.plot(sthovl, mff_jl_rad[:, 2], ":", label=f"<j2> {d_info['Type symbol']:} {d_info['Shell']:}", alpha=0.7)
                    if ("mff_j0_tab" in d_info.keys()):
                        ax.plot(d_info["sthovl"], d_info["mff_j0_tab"], "-", label=f"<j0> {d_info['Type symbol']:} tab.", alpha=0.7)
                    if ("mff_j2_tab" in d_info.keys()):
                        ax.plot(d_info["sthovl"], d_info["mff_j2_tab"], ":", label=f"<j2> {d_info['Type symbol']:} tab.", alpha=0.7)
                ax.set_xlim((0., 2.))
                ax.legend()
                plt.show()
    
    button = Button(description="Get Info")
    button.on_click(get_info_about_elements)
    page_handbook = widgets.Box(
        [tags, button], 
        layout=Layout(display='flex',
                      flex_flow='column',
                      width="100%")
                      )
    return page_handbook


In [17]:
page_menu = draw_page_menu()
page_examples = draw_page_examples()
page_object, SELECT_DATA_ITEM, SELECT_LOOP_ITEM, CIF_TEXT = draw_page_object()
page_cryspy_methods = draw_page_cryspy_methods()
page_handbook = draw_page_handbook()

In [ ]:
ACCORDION = widgets.Accordion(
            children=[page_handbook, 
            widgets.VBox([page_menu, widgets.Label("Examples:"), page_examples]), 
            page_object, ],
            titles=('Handbook', "Upload File", "RCIF File")
            )

In [ ]:
accordion_methods = widgets.Accordion(
    children=[page_cryspy_methods,],
    titles = ("RhoChi and MEMPy procedures", )
    )

In [24]:
# tab_1 = widgets.VBox(
#     [page_menu, page_examples, page_object, page_cryspy_methods]
# )

# tab = widgets.Tab(layout=Layout(display='flex',
#                       flex_flow='raw', flex_grow="1"))
# tab.children = [tab_1, page_handbook]
# tab.titles = ['Working with RCIF', 'Use as a handbook']

# tab.observe(lambda x: W_OUT.clear_output(), names="selected_index")


In [17]:
display(ACCORDION, accordion_methods, W_OUT)

1


In [19]:
def DISPLAY_NEW_OBJECT():
    l_data_item_name = [item.get_name() for item in DDATA["rcif_obj"].items]
    
    SELECT_DATA_ITEM.options = l_data_item_name
    # if (SELECT_DATA_ITEM.index is None) and len(l_data_item_name)>0:
    #     SELECT_DATA_ITEM.index = 0
    ACCORDION.selected_index = 2
    if (SELECT_DATA_ITEM.index is None):
        return
    
    data_index = SELECT_DATA_ITEM.index
    if data_index >= len(l_data_item_name):
        data_index = 0
         
    data_item = DDATA["rcif_obj"].items[data_index]
    if isinstance(data_item, cryspy.DataN):
        l_loop_item_name = [item.get_name() for item in data_item.items]
        SELECT_LOOP_ITEM.options = l_loop_item_name

        # if (SELECT_LOOP_ITEM.index is None) and len(l_loop_item_name)>0:
        #     SELECT_LOOP_ITEM.index = 0
        if (SELECT_LOOP_ITEM.index is None):
            return
        loop_index = SELECT_LOOP_ITEM.index
        if loop_index >= len(l_loop_item_name):
            loop_index = 0
        loop_item = data_item.items[0]
        CIF_TEXT.value = loop_item.to_cif()
    else:
        CIF_TEXT.value = data_item.to_cif()
        